In [1]:
import pandas as pd
import numpy as np
import wget
import zipfile
import os

if not os.path.isfile("rawinventor.tsv"):
    wget.download("https://s3.amazonaws.com/data.patentsview.org/download/rawinventor.tsv.zip")
    with zipfile.ZipFile("rawinventor.tsv.zip", 'r') as zip_ref:
        zip_ref.extractall(".")
    os.remove("rawinventor.tsv.zip")

In [2]:
rawinventor = pd.read_csv("rawinventor.tsv", sep="\t", dtype=str)
data = pd.read_csv("input/combined_labels_HarvardInv.txt", sep="|", dtype=str)
data

,Pub_number,ManualIDs,OurIDs,LiLow,LiHigh,RawNames
0,US05712011,"277,488,587","HI203842,HI220403,HM245454","04307879-1,04843112-2,05545128-2","04307879-1,04843112-2,05545128-2","HAYES WILSON C,MCMAHON THOMAS A,ROBINOVITCH ST..."
1,US05720935,"204,3,352","HM29369,LI1212917,LI506629","04534483-1,05077034-2,05574148-2","04534483-1,05077034-2,05574148-2","ADELSTEIN S JAMES,FOULON CATHERINE F,KASSIS AM..."
2,US05721106,"466,472","HI468617,HM259155","05434050-1,05434050-2","05434050-1,05434050-2","MAGGIO JOHN E,MANTYH PATRICK W"
3,US05721362,"133,229","HI1616852,HI73623","03867460-1,05721362-2","03867460-1,05721362-2","COREY ELIAS J,GIN DAVID"
4,US05726025,"363,365,554","HI230747,HM103773,HM49294","05573944-1,05654150-1,05726025-3","05573944-1,05654150-1,05726025-3","KING RANDALL W,KIRSCHNER MARC W,PETERS JAN MIC..."
...,...,...,...,...,...,...
581,US07846738,"226,238,424,673","HI1076972,HI32971,HM159290,HM189667","04154625-1,06464842-5,07846738-2,07846738-3","04154625-1,06464842-5,07846738-2,07846738-3","GERSHOW MARC H,GOLOVCHENKO JENE A,LI JIALI,STE..."
582,US07846962,"105,183,265,518,9","HI15155,HI336536,HM244685,HM311780,HM355662","05358959-1,07256197-1,07737172-1,07737172-2,07...","05358959-1,07256197-1,07737172-1,07737172-2,07...","AKTAS HUSEYIN,CHEN HAN,FAN YUN HUA,HALPERIN JO..."
583,US07851658,"343,442,596","HI21536,HM116542,LI1273435","07045337-4,07070928-2,07851658-2","07045337-4,07070928-2,07851658-2","KANAN MATTHEW W,LIU DAVID R,ROZENMAN MARY M"
584,US07858965,"434,455,705,773,778,782","HI1077482,HI10858,HM153990,HM233255,LI878627,L...","05196396-1,06765448-1,07858965-0,07858965-1,07...","05196396-1,06765448-1,07858965-0,07858965-1,07...","LIEBER CHARLES M,LU WEI,TIMKO BRIAN P,WU YUE,X..."


In [3]:
patent_id = data.apply(lambda x: [x["Pub_number"]]*len(str.split(x["ManualIDs"], ",")), axis=1)
inventor_ids = data.ManualIDs.str.split(",")
sequence = data.apply(lambda x: list(range(len(str.split(x["ManualIDs"], ",")))), axis=1)

assert all(sequence.apply(lambda x: len(x)) == inventor_ids.apply(lambda x: len(x)))
assert all(sequence.apply(lambda x: len(x)) == patent_id.apply(lambda x: len(x)))

In [6]:
cleaned = pd.concat({"patent_id":patent_id.explode(), "inventor_id":inventor_ids.explode(), "sequence":sequence.explode()}, axis=1)
cleaned["patent_number"] = cleaned.patent_id.apply(lambda x: x[2:]).str.lstrip("0")
cleaned["mention_id"] = "US" + cleaned.patent_number + "-" + cleaned.sequence

,patent_id,inventor_id,sequence,patent_number
0,US05712011,277,0,5712011
0,US05712011,488,1,5712011
0,US05712011,587,2,5712011
1,US05720935,204,0,5720935
1,US05720935,3,1,5720935
...,...,...,...,...
584,US07858965,705,2,7858965
584,US07858965,773,3,7858965
584,US07858965,778,4,7858965
584,US07858965,782,5,7858965
